In [1]:
import pandas as pd
import re

In [2]:
inp_file = './results_spert_tr_filtered_4NER_xlmroberta/all_folds.out'

In [3]:
#если несколько фолдов
from os.path import dirname, realpath, splitext, basename

folds_sep = '(?<=\n)\d+\n'
with open(inp_file, 'r') as f:
    txt = f.read()
    
seps = '\n\d\n|^\d+\n'
fold_nums = re.findall(seps, txt)
fold_nums = list(map(lambda x: re.sub('\D', '', x), fold_nums))


txt_parts = re.split(folds_sep, txt)
txt_parts = [txt_part for txt_part in txt_parts if txt_part]
if len(txt_parts)!=len(fold_nums):
    if txt_parts[0].find('Evaluation')<0:
        txt_parts = txt_parts[1:]
    elif txt_parts[-1].find('Evaluation')<0:
        txt_parts = txt_parts[:-1]
    else:
        raise AssertionError
    assert len(txt_parts)==len(fold_nums)

filename = splitext(basename(inp_file))[0]
for i, fold in enumerate(fold_nums):
    fold_filename = fold + '_' + filename + '.out'
    with open(dirname(realpath(inp_file)) + '/' + fold_filename, 'w') as f:
        f.write(txt_parts[i])

In [4]:
#если несколько фолдов
import os

for file in os.listdir(dirname(realpath(inp_file))):
    if not file[0].isdigit():
        continue
    fold_num = file.split('_')[0]
    #читаем файл, делим на сперт репорты
    with open(dirname(realpath(inp_file)) + '/' + file, 'r') as f:
        txt = f.read()
    txt_parts = re.split('Evaluation\n\n|--------------------------------------------------', txt)
    txt_parts = [txt_part for txt_part in txt_parts if txt_part]
    spert_reports = [txt_part for txt_part in txt_parts if re.search('--- Entities .+\s+---', txt_part)]

    spert_reports = [re.sub('\n\s+', '\n', rep) for rep in spert_reports]


    #основной код
    def count_symbols_before_first_whitespace(matchobj):
        num_symbols = matchobj.span()[0]
        symbols_before_first_whitespace.append(num_symbols)
        return '<'+str(num_symbols)+'>'

    def make_valid_margin(matchobj):
        margin = max_margin - int(matchobj.group(0)[1:-1])
        return ' '*margin

    def delete_n_first_whitespaces(matchobj):
        columns = matchobj.group()
        first_n_whitespaces = re.match('\s+', columns).span()[1]
        return ' '*(first_n_whitespaces-delete_n_first_whitespaces.n)+columns[first_n_whitespaces:]



    for rep_id in range(len(spert_reports)):
        sents = re.split('\n+', spert_reports[rep_id])
        #sents = spert_reports[rep_id].split('\n')
        symbols_before_first_whitespace = []
        for sent_id in range(1, len(sents)):
            #считаем кол-во символов до первого пробела в каждом предложении, 
            #заменяем первую череду пробелов в предложении
            #на это кол-во символов
            if sents[sent_id].find('---')>=0 or re.search('\d\.\d', sents[sent_id]) is None and sents[sent_id].find('f1-score')<0:
                sents[sent_id] = '\n' + sents[sent_id]
                continue
            if sents[sent_id].find('MainThread')>=0:
                break
            sents[sent_id] = re.sub('\s+', count_symbols_before_first_whitespace, sents[sent_id], count=1)
            sents[sent_id] = re.sub('(?<=\d)\s+', ' '*11, sents[sent_id], count=1)
            #след. строчка для того, чтобы меньше 4 цифр через запятую 
            #(любые 3 цифры, например 0.00, 9.40 и т.д.) 
            #норм обрабатывались
            sents[sent_id] = re.sub('(?<=\W\d\.\d\d)\s+', ' '*12, sents[sent_id], count=1)
            delete_n_first_whitespaces.n=2
            sents[sent_id] = re.sub('\s+\d+\.\d+\s+\d+$', delete_n_first_whitespaces, sents[sent_id])
            sents[sent_id] = re.sub('\s+(\d+)$', ' '*9+r'\1', sents[sent_id])
            #след. строчка для того, чтобы меньше 4 цифр через запятую (3 цифры, например
            #(любые 3 цифры, например 0.00, 9.40 и т.д.) 
            #норм обрабатывались
            sents[sent_id] = re.sub('(?<=\W\d\.\d\d)\s+(\d+)$', ' '*10+r'\1', sents[sent_id])

        max_margin = max(symbols_before_first_whitespace)+4
        spert_reports[rep_id] = '\n'.join(sents)
        spert_reports[rep_id] = re.sub('<\d+>', make_valid_margin, spert_reports[rep_id])

    out_txt = spert_reports[0]
    out_file = dirname(realpath(inp_file)) + '/' + 'pretty_' + file
    with open(out_file, 'w') as f:
        f.write(out_txt)

IndexError: list index out of range

In [5]:
#если несколько фолдов
#сделаем таблички, а также агрегируем точности
import pandas as pd

NER_res_dfs = []
RE_gold_NER_dfs = []
RE_pred_NER_dfs = []

for file in os.listdir(dirname(realpath(inp_file))):
    if file.find('pretty_')<0:
        continue
    with open(dirname(realpath(inp_file)) + '/' + file, 'r') as f:
        txt = f.read()
    file = file.replace('pretty_', '')
    fold_num = file.split('_')[0]
    file = re.sub('\d+_', '', file)
    all_eval_parts = re.split('---\sEntities\s(named\sentity\srecognition\s(NER))\s---|---\sRelations\s---', txt)
    NER_res, RE_pred_NER = [part for part in all_eval_parts if part]
    #NER
    NER_res= [row for row in NER_res.split('\n') if row]
    NER_res = NER_res[2:]
    col_names = re.split('\s+', NER_res[0])[1:]
    NER_df =  pd.DataFrame(columns=col_names)
    for row in NER_res[1:]:
        row = re.split('\s+', row)
        NER_df = NER_df.append(pd.Series(map(float, row[1:]), index=col_names, name=row[0].replace('none_', '')))
    NER_df['support'] = NER_df['support'].apply(int)
    NER_res_dfs.append(NER_df)
    out_file = realpath(dirname(realpath(inp_file))) + '/' + fold_num + '_' + 'NER' + '_' + file.replace('.out', '.csv')
    NER_df.to_csv(out_file)
    #RE pred NER
    RE_pred_NER = RE_pred_NER.split('A relation is considered correct if the relation type and the two related entities are predicted correctly (in span and entity type)')[-1]
    RE_pred_NER = re.sub('Parse.+\n', '', RE_pred_NER)
    RE_pred_NER = [row for row in RE_pred_NER.split('\n') if row]
    RE_pred_NER = RE_pred_NER[:-2]
    col_names = re.split('\s+', RE_pred_NER[0])[1:]
    rel_df =  pd.DataFrame(columns=col_names)
    for row in RE_pred_NER[1:]:
        row = re.split('\s+', row)
        rel_df = rel_df.append(pd.Series(map(float, row[1:]), index=col_names, name=row[0].replace('none_', '')))
    rel_df['support'] = rel_df['support'].apply(int)
    RE_pred_NER_dfs.append(rel_df)
    out_file = realpath(dirname(realpath(inp_file))) + '/' + fold_num + '_' + 'RE_pred_NER' + '_' + file.replace('.out', '.csv')
    rel_df.to_csv(out_file)

ValueError: could not convert string to float: 'precision'

In [6]:
#создаем одну общую таблицу

Только для 5ого фолда комплексного теста!!!!

In [7]:
RE_pred_NER_dfs[-1].at['Diseasename_Indication_0', 'f1-score'] = float(0)

In [7]:
RE_f1_pred_NER_dfs = list(map(lambda x: pd.DataFrame(x['f1-score']),RE_pred_NER_dfs))

In [7]:
RE_f1_pred_NER_dfs

[                           f1-score
 Diseasename_Indication_1      49.54
 Drugname_SourceInfodrug_0     48.26
 Drugname_Diseasename_1        73.56
 ADR_Drugname_0                22.22
 ADR_Drugname_1                50.57
 Drugname_SourceInfodrug_1     52.39
 Diseasename_Indication_0      10.00
 Drugname_Diseasename_0        55.16,
                            f1-score
 ADR_Drugname_1                42.04
 Drugname_SourceInfodrug_0     42.08
 Drugname_Diseasename_0        49.74
 Drugname_Diseasename_1        76.16
 Diseasename_Indication_1      50.57
 ADR_Drugname_0                37.50
 Drugname_SourceInfodrug_1     50.67
 Diseasename_Indication_0      10.73,
                            f1-score
 Diseasename_Indication_0      12.84
 ADR_Drugname_1                42.53
 Drugname_SourceInfodrug_0     33.05
 Drugname_SourceInfodrug_1     46.77
 ADR_Drugname_0                42.82
 Diseasename_Indication_1      53.76
 Drugname_Diseasename_0        50.60
 Drugname_Diseasename_1        76.07

In [13]:
import json
with open('./data/RDRS_multicontext_train_filtered/folds/5/complex_test.json', 'r') as f:
    data = json.load(f)

rel_set = set()
for rev in data:
    for rel in rev['relations']:
        rel_set.add(rel['relation_type'])
print(rel_set)

{'none_ADR_Drugname', 'Drugname_SourceInfodrug', 'Drugname_Diseasename', 'none_Drugname_SourceInfodrug', 'none_Drugname_Diseasename', 'Diseasename_Indication', 'ADR_Drugname'}


In [8]:
import numpy as np

avg_RE_pred_NER_df = sum(RE_f1_pred_NER_dfs)/len(RE_f1_pred_NER_dfs)
avg_RE_pred_NER_df['std']=float(0)
for rel_type in avg_RE_pred_NER_df.index:
    avg_RE_pred_NER_df.at[rel_type, 'std'] = np.round(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_pred_NER_dfs))), 3)

Только для 5ого фолда комплексного теста!!!!

In [10]:
avg_RE_pred_NER_df.at['Diseasename_Indication_0', 'std'] = np.round(np.std(list(map(lambda x: x['f1-score']['Diseasename_Indication_0'], RE_pred_NER_dfs[:-1]))), 3)
avg_RE_pred_NER_df.at['Diseasename_Indication_0', 'f1-score'] = np.round(np.mean(list(map(lambda x: x['f1-score']['Diseasename_Indication_0'], RE_pred_NER_dfs[:-1]))), 2)

In [9]:
avg_RE_pred_NER_df

,f1-score,std
ADR_Drugname_0,30.382,9.159
ADR_Drugname_1,38.272,8.849
Diseasename_Indication_0,11.690,2.234
Diseasename_Indication_1,49.568,2.569
Drugname_Diseasename_0,53.156,4.471
Drugname_Diseasename_1,74.654,2.095
Drugname_SourceInfodrug_0,40.060,5.178
Drugname_SourceInfodrug_1,50.170,2.803


In [9]:
avg_RE_pred_NER_df

,f1-score,std
ADR_Drugname_0,48.070,3.526
ADR_Drugname_1,52.744,4.445
Diseasename_Indication_0,10.632,7.451
Diseasename_Indication_1,52.120,7.718
Drugname_Diseasename_0,61.864,3.519
Drugname_Diseasename_1,77.896,2.092
Drugname_SourceInfodrug_0,40.090,6.425
Drugname_SourceInfodrug_1,49.970,3.653


In [105]:
avg_RE_pred_NER_df

,f1-score,std
ADR_Drugname_0,30.382,9.159
ADR_Drugname_1,38.272,8.849
Diseasename_Indication_0,11.690,2.234
Diseasename_Indication_1,49.568,2.569
Drugname_Diseasename_0,53.156,4.471
Drugname_Diseasename_1,74.654,2.095
Drugname_SourceInfodrug_0,40.060,5.178
Drugname_SourceInfodrug_1,50.170,2.803


In [12]:
out_file = realpath(dirname(realpath(inp_file))) + '/avg_RE_pred_NER_df.csv'
avg_RE_pred_NER_df.to_csv(out_file)

In [10]:
NER_avg_df = sum(NER_res_dfs)/len(NER_res_dfs)
NER_avg_df = NER_avg_df.drop('support', 1)

NER_avg_df['std (for f1_score)'] = float(0)
for rel_type in NER_avg_df.index:
    NER_avg_df.at[rel_type, 'std (for f1_score)'] = float(np.round(np.std(list(map(lambda x: x['f1-score'][rel_type], NER_res_dfs))), 3))

NER_avg_df

,precision,recall,f1-score,std (for f1_score)
ADR,44.810,41.438,42.868,5.694
Disease:DisTypeDiseasename,82.876,86.562,84.646,2.740
Disease:DisTypeIndication,57.428,67.094,61.878,2.705
Medication:MedTypeDrugname,92.752,96.510,94.586,0.581
Medication:MedTypeSourceInfodrug,49.384,71.200,58.268,4.367
macro,65.448,72.560,68.448,1.908
micro,74.058,81.506,77.598,1.302


In [11]:
out_file = realpath(dirname(realpath(inp_file))) + '/avg_NER_df.csv'
NER_avg_df.to_csv(out_file)

In [15]:
#flatten df
sorted_rel_types = ['ADR_Drugname', 'Drugname_Diseasename', 'Drugname_SourceInfodrug', 'Diseasename_Indication']
pos_neg_order = ['0', '1']

sorted_rel_types = [[rel_type + '_' + rel_class for rel_class in pos_neg_order] for rel_type in sorted_rel_types]
sorted_rel_types = sum(sorted_rel_types, [])
flatten_RE_df = pd.DataFrame({k: [float(0)] for k in sorted_rel_types})
for rel_type in sorted_rel_types:
    flatten_RE_df[rel_type][0] = np.round(avg_RE_pred_NER_df['f1-score'][rel_type], 2)
out_file = realpath(dirname(realpath(inp_file))) + '/avg_flattened_pred_NER_RE_df.csv'
flatten_RE_df.to_csv(out_file, index=False)
flatten_RE_df

,ADR_Drugname_0,ADR_Drugname_1,Drugname_Diseasename_0,Drugname_Diseasename_1,Drugname_SourceInfodrug_0,Drugname_SourceInfodrug_1,Diseasename_Indication_0,Diseasename_Indication_1
0,33.37,41.78,57.64,74.4,24.42,34.04,2.86,45.89


In [118]:
#flatten df
sorted_rel_types = ['ADR_Drugname', 'Drugname_Diseasename', 'Drugname_SourceInfodrug', 'Diseasename_Indication']
pos_neg_order = ['0', '1']

sorted_rel_types = [[rel_type + '_' + rel_class for rel_class in pos_neg_order] for rel_type in sorted_rel_types]
sorted_rel_types = sum(sorted_rel_types, [])
flatten_RE_df = pd.DataFrame({k: [float(0)] for k in sorted_rel_types})
for rel_type in sorted_rel_types:
    flatten_RE_df[rel_type][0] = np.round(avg_RE_pred_NER_df['f1-score'][rel_type], 2)
out_file = realpath(dirname(realpath(inp_file))) + '/avg_flattened_pred_NER_RE_df.csv'
flatten_RE_df.to_csv(out_file, index=False)
flatten_RE_df

,ADR_Drugname_0,ADR_Drugname_1,Drugname_Diseasename_0,Drugname_Diseasename_1,Drugname_SourceInfodrug_0,Drugname_SourceInfodrug_1,Diseasename_Indication_0,Diseasename_Indication_1
0,30.55,38.69,53.49,74.85,42.29,53.03,11.69,50.4


In [63]:
flatten_RE_df

,ADR_Drugname_0,ADR_Drugname_1,Drugname_Diseasename_0,Drugname_Diseasename_1,Drugname_SourceInfodrug_0,Drugname_SourceInfodrug_1,Diseasename_Indication_0,Diseasename_Indication_1


In [92]:
inp_file

'./results_joint/RDRS_all_folds_2382077.out'

In [82]:
print(NER_test_res[2])

A


In [311]:
#читаем файл, делим на сперт репорты
with open(inp_file, 'r') as f:
    txt = f.read()

txt_parts = re.split('Evaluation\n\n|--------------------------------------------------', txt)
txt_parts = [txt_part for txt_part in txt_parts if txt_part]
spert_reports = [txt_part for txt_part in txt_parts if re.search('--- Entities .+\s+---', txt_part)]

spert_reports = [re.sub('\n\s+', '\n', rep) for rep in spert_reports]


#основной код
def count_symbols_before_first_whitespace(matchobj):
    num_symbols = matchobj.span()[0]
    symbols_before_first_whitespace.append(num_symbols)
    return '<'+str(num_symbols)+'>'

def make_valid_margin(matchobj):
    margin = max_margin - int(matchobj.group(0)[1:-1])
    return ' '*margin

def delete_n_first_whitespaces(matchobj):
    columns = matchobj.group()
    first_n_whitespaces = re.match('\s+', columns).span()[1]
    return ' '*(first_n_whitespaces-delete_n_first_whitespaces.n)+columns[first_n_whitespaces:]



for rep_id in range(len(spert_reports)):
    sents = re.split('\n+', spert_reports[rep_id])
    #sents = spert_reports[rep_id].split('\n')
    symbols_before_first_whitespace = []
    for sent_id in range(1, len(sents)):
        #считаем кол-во символов до первого пробела в каждом предложении, 
        #заменяем первую череду пробелов в предложении
        #на это кол-во символов
        if sents[sent_id].find('---')>=0 or re.search('\d\.\d', sents[sent_id]) is None and sents[sent_id].find('f1-score')<0:
            sents[sent_id] = '\n' + sents[sent_id]
            continue
        if sents[sent_id].find('MainThread')>=0:
            break
        sents[sent_id] = re.sub('\s+', count_symbols_before_first_whitespace, sents[sent_id], count=1)
        sents[sent_id] = re.sub('(?<=\d)\s+', ' '*11, sents[sent_id], count=1)
        #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
        sents[sent_id] = re.sub('(?<=\W0\.00)\s+', ' '*12, sents[sent_id], count=1)
        delete_n_first_whitespaces.n=2
        sents[sent_id] = re.sub('\s+\d+\.\d+\s+\d+$', delete_n_first_whitespaces, sents[sent_id])
        sents[sent_id] = re.sub('\s+(\d+)$', ' '*9+r'\1', sents[sent_id])
        #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
        sents[sent_id] = re.sub('(?<=\W0\.00)\s+(\d+)$', ' '*10+r'\1', sents[sent_id])
        
    max_margin = max(symbols_before_first_whitespace)+4
    spert_reports[rep_id] = '\n'.join(sents)
    spert_reports[rep_id] = re.sub('<\d+>', make_valid_margin, spert_reports[rep_id])

#дампим вывод
out_file = './results/pretty_print_positive_relations.txt'


txt = '============\n'.join(spert_reports)
with open(out_file, 'w') as f:
    f.write(txt)

In [ ]:
echo "1"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext/1/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext/1/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/1/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/1/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext/1/"

In [73]:
sh = '''
#! /bin/sh

#SBATCH -D /s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src
#SBATCH -o RDRS_simple_all_folds_%j.out
#SBATCH -e RDRS_simple_all_folds_%j.err
#SBATCH -p hpc5-v100-3d
#SBATCH -n 1
#SBATCH --gres=gpu:v100:1

#TODO:
# Переименовать файлы, чтобы понятные названия были
# Параметры командной строки, а не ворох строк в файле

source /s/ls4/users/romanrybka/anaconda_relations/bin/activate
conda activate /s/ls4/users/romanrybka/anaconda_relations/envs/relation_extraction_pure

#путь до корневой папки проекта
echo "1"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/1/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/1/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/1/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/1/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext_spert_valid_reviews/1/"


#================NER====================
mkdir -p $RES_NER_MODEL_DIR
mkdir -p $PRED_DIR/NER

# обучение модели. Конфиги вбивайте сами
python $PYTHONPATH/ner/train_model.py $TRAIN_JSON $NER_MODEL_PATH $RES_NER_MODEL_DIR --lr 0.00002 --n_epochs 10 --max_seq_length 512 --batch_size 8

# предсказание на обучающем и тестировочном множестве (TODO: предсказание на валиде)
python ner/predict.py $TRAIN_JSON $PRED_DIR/NER/ner_train_pred.json  $RES_NER_MODEL_DIR
python ner/predict.py $TEST_JSON $PRED_DIR/NER/ner_test_pred.json $RES_NER_MODEL_DIR

# оценка предсказаний
python ner/evaluate.py $PRED_DIR/NER/ner_train_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_train_eval.json
python ner/evaluate.py $PRED_DIR/NER/ner_test_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_test_eval.json

#==================RE====================
mkdir -p $PRED_DIR/RE

# Подготовка данных
python re/prepare_ds.py $TRAIN_JSON $PRED_DIR/RE/re_train_data.tsv --mode NER_fixed --save_label_map
python re/prepare_ds.py $TEST_JSON $PRED_DIR/RE/re_test_data_ner_gold.tsv --mode NER_fixed --label_map_path $PRED_DIR/RE/label_map.json

python re/prepare_ds.py $PRED_DIR/NER/ner_test_pred.json $PRED_DIR/RE/re_test_data_ner_predicted.tsv --mode Joint --label_map_path $PRED_DIR/RE/label_map.json


# Обучение RE-модели
python re/fit.py $PRED_DIR/RE/re_train_data.tsv $PRED_DIR/RE/re_train_data.tsv --model_path $RE_MODEL_PATH --model_type $RE_MODEL_TYPE --model_out_path $RES_RE_MODEL_DIR --args_path $RE_MODEL_ARGS

# Предсказания на NER-gold
python re/predict.py $PRED_DIR/RE/re_test_data_ner_gold.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv

# Предсказания на NER-pred
python re/predict.py $PRED_DIR/RE/re_test_data_ner_predicted.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv

# # Постпроцессинг предсказаний (перевод в .json для оценки)
# Gold
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv --origin_json_file_path $TEST_JSON --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_gold_ner.json

# Pred
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv --origin_json_file_path $PRED_DIR/RE/generated_potential_relations.json --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_pred_ner.json

# Оценка на NER-gold
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_gold_ner.json -o $PRED_DIR/RE/gold_RE_eval_res_
# Оценка на NER-pred
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_pred_ner.json -o $PRED_DIR/RE/pred_RE_eval_res_


# путь до корневой папки проекта
echo "2"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/2/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/2/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/2/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/2/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext_spert_valid_reviews/2/"


#================NER====================
mkdir -p $RES_NER_MODEL_DIR
mkdir -p $PRED_DIR/NER

# обучение модели. Конфиги вбивайте сами
python $PYTHONPATH/ner/train_model.py $TRAIN_JSON $NER_MODEL_PATH $RES_NER_MODEL_DIR --lr 0.00002 --n_epochs 10 --max_seq_length 512 --batch_size 8

# предсказание на обучающем и тестировочном множестве (TODO: предсказание на валиде)
python ner/predict.py $TRAIN_JSON $PRED_DIR/NER/ner_train_pred.json  $RES_NER_MODEL_DIR
python ner/predict.py $TEST_JSON $PRED_DIR/NER/ner_test_pred.json $RES_NER_MODEL_DIR

# оценка предсказаний
python ner/evaluate.py $PRED_DIR/NER/ner_train_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_train_eval.json
python ner/evaluate.py $PRED_DIR/NER/ner_test_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_test_eval.json

#==================RE====================
mkdir -p $PRED_DIR/RE

# Подготовка данных
python re/prepare_ds.py $TRAIN_JSON $PRED_DIR/RE/re_train_data.tsv --mode NER_fixed --save_label_map
python re/prepare_ds.py $TEST_JSON $PRED_DIR/RE/re_test_data_ner_gold.tsv --mode NER_fixed --label_map_path $PRED_DIR/RE/label_map.json

python re/prepare_ds.py $PRED_DIR/NER/ner_test_pred.json $PRED_DIR/RE/re_test_data_ner_predicted.tsv --mode Joint --label_map_path $PRED_DIR/RE/label_map.json


# Обучение RE-модели
python re/fit.py $PRED_DIR/RE/re_train_data.tsv $PRED_DIR/RE/re_train_data.tsv --model_path $RE_MODEL_PATH --model_type $RE_MODEL_TYPE --model_out_path $RES_RE_MODEL_DIR --args_path $RE_MODEL_ARGS

# Предсказания на NER-gold
python re/predict.py $PRED_DIR/RE/re_test_data_ner_gold.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv

# Предсказания на NER-pred
python re/predict.py $PRED_DIR/RE/re_test_data_ner_predicted.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv

# # Постпроцессинг предсказаний (перевод в .json для оценки)
# Gold
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv --origin_json_file_path $TEST_JSON --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_gold_ner.json

# Pred
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv --origin_json_file_path $PRED_DIR/RE/generated_potential_relations.json --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_pred_ner.json

# Оценка на NER-gold
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_gold_ner.json -o $PRED_DIR/RE/gold_RE_eval_res_
# Оценка на NER-pred
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_pred_ner.json -o $PRED_DIR/RE/pred_RE_eval_res_


# путь до корневой папки проекта
echo "3"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/3/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/3/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/3/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/3/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext_spert_valid_reviews/3/"


#================NER====================
mkdir -p $RES_NER_MODEL_DIR
mkdir -p $PRED_DIR/NER

# обучение модели. Конфиги вбивайте сами
python $PYTHONPATH/ner/train_model.py $TRAIN_JSON $NER_MODEL_PATH $RES_NER_MODEL_DIR --lr 0.00002 --n_epochs 10 --max_seq_length 512 --batch_size 8

# предсказание на обучающем и тестировочном множестве (TODO: предсказание на валиде)
python ner/predict.py $TRAIN_JSON $PRED_DIR/NER/ner_train_pred.json  $RES_NER_MODEL_DIR
python ner/predict.py $TEST_JSON $PRED_DIR/NER/ner_test_pred.json $RES_NER_MODEL_DIR

# оценка предсказаний
python ner/evaluate.py $PRED_DIR/NER/ner_train_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_train_eval.json
python ner/evaluate.py $PRED_DIR/NER/ner_test_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_test_eval.json

#==================RE====================
mkdir -p $PRED_DIR/RE

# Подготовка данных
python re/prepare_ds.py $TRAIN_JSON $PRED_DIR/RE/re_train_data.tsv --mode NER_fixed --save_label_map
python re/prepare_ds.py $TEST_JSON $PRED_DIR/RE/re_test_data_ner_gold.tsv --mode NER_fixed --label_map_path $PRED_DIR/RE/label_map.json

python re/prepare_ds.py $PRED_DIR/NER/ner_test_pred.json $PRED_DIR/RE/re_test_data_ner_predicted.tsv --mode Joint --label_map_path $PRED_DIR/RE/label_map.json


# Обучение RE-модели
python re/fit.py $PRED_DIR/RE/re_train_data.tsv $PRED_DIR/RE/re_train_data.tsv --model_path $RE_MODEL_PATH --model_type $RE_MODEL_TYPE --model_out_path $RES_RE_MODEL_DIR --args_path $RE_MODEL_ARGS

# Предсказания на NER-gold
python re/predict.py $PRED_DIR/RE/re_test_data_ner_gold.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv

# Предсказания на NER-pred
python re/predict.py $PRED_DIR/RE/re_test_data_ner_predicted.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv

# # Постпроцессинг предсказаний (перевод в .json для оценки)
# Gold
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv --origin_json_file_path $TEST_JSON --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_gold_ner.json

# Pred
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv --origin_json_file_path $PRED_DIR/RE/generated_potential_relations.json --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_pred_ner.json

# Оценка на NER-gold
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_gold_ner.json -o $PRED_DIR/RE/gold_RE_eval_res_
# Оценка на NER-pred
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_pred_ner.json -o $PRED_DIR/RE/pred_RE_eval_res_


# путь до корневой папки проекта
echo "4"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/4/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/4/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/4/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/4/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext_spert_valid_reviews/4/"


#================NER====================
mkdir -p $RES_NER_MODEL_DIR
mkdir -p $PRED_DIR/NER

# обучение модели. Конфиги вбивайте сами
python $PYTHONPATH/ner/train_model.py $TRAIN_JSON $NER_MODEL_PATH $RES_NER_MODEL_DIR --lr 0.00002 --n_epochs 10 --max_seq_length 512 --batch_size 8

# предсказание на обучающем и тестировочном множестве (TODO: предсказание на валиде)
python ner/predict.py $TRAIN_JSON $PRED_DIR/NER/ner_train_pred.json  $RES_NER_MODEL_DIR
python ner/predict.py $TEST_JSON $PRED_DIR/NER/ner_test_pred.json $RES_NER_MODEL_DIR

# оценка предсказаний
python ner/evaluate.py $PRED_DIR/NER/ner_train_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_train_eval.json
python ner/evaluate.py $PRED_DIR/NER/ner_test_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_test_eval.json

#==================RE====================
mkdir -p $PRED_DIR/RE

# Подготовка данных
python re/prepare_ds.py $TRAIN_JSON $PRED_DIR/RE/re_train_data.tsv --mode NER_fixed --save_label_map
python re/prepare_ds.py $TEST_JSON $PRED_DIR/RE/re_test_data_ner_gold.tsv --mode NER_fixed --label_map_path $PRED_DIR/RE/label_map.json

python re/prepare_ds.py $PRED_DIR/NER/ner_test_pred.json $PRED_DIR/RE/re_test_data_ner_predicted.tsv --mode Joint --label_map_path $PRED_DIR/RE/label_map.json


# Обучение RE-модели
python re/fit.py $PRED_DIR/RE/re_train_data.tsv $PRED_DIR/RE/re_train_data.tsv --model_path $RE_MODEL_PATH --model_type $RE_MODEL_TYPE --model_out_path $RES_RE_MODEL_DIR --args_path $RE_MODEL_ARGS

# Предсказания на NER-gold
python re/predict.py $PRED_DIR/RE/re_test_data_ner_gold.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv

# Предсказания на NER-pred
python re/predict.py $PRED_DIR/RE/re_test_data_ner_predicted.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv

# # Постпроцессинг предсказаний (перевод в .json для оценки)
# Gold
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv --origin_json_file_path $TEST_JSON --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_gold_ner.json

# Pred
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv --origin_json_file_path $PRED_DIR/RE/generated_potential_relations.json --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_pred_ner.json

# Оценка на NER-gold
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_gold_ner.json -o $PRED_DIR/RE/gold_RE_eval_res_
# Оценка на NER-pred
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_pred_ner.json -o $PRED_DIR/RE/pred_RE_eval_res_


# путь до корневой папки проекта
echo "5"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/5/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/5/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# путь сохранения модели после fine-tuning'a
export RES_NER_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/5/model_ner/"

# Путь к изначальной языковой модели
export RE_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-large_sag_1ep"
# Путь для сохранения RE-модели после fine-tuning'а 
export RES_RE_MODEL_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/outputs/5/model_re/"
# Тип RE-модели
export RE_MODEL_TYPE="xlmroberta"
# Путь к конфигу обучения RE-модели
export RE_MODEL_ARGS="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/configs/roberta-large-sag.json"

# путь, куда будут сохранены файлы .csv и .json с предсказаниями, а также все промежуточные файлы, которые получаются в процессе работы пайплайна
export PRED_DIR="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext_spert_valid_reviews/5/"


#================NER====================
mkdir -p $RES_NER_MODEL_DIR
mkdir -p $PRED_DIR/NER

# обучение модели. Конфиги вбивайте сами
python $PYTHONPATH/ner/train_model.py $TRAIN_JSON $NER_MODEL_PATH $RES_NER_MODEL_DIR --lr 0.00002 --n_epochs 10 --max_seq_length 512 --batch_size 8

# предсказание на обучающем и тестировочном множестве (TODO: предсказание на валиде)
python ner/predict.py $TRAIN_JSON $PRED_DIR/NER/ner_train_pred.json  $RES_NER_MODEL_DIR
python ner/predict.py $TEST_JSON $PRED_DIR/NER/ner_test_pred.json $RES_NER_MODEL_DIR

# оценка предсказаний
python ner/evaluate.py $PRED_DIR/NER/ner_train_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_train_eval.json
python ner/evaluate.py $PRED_DIR/NER/ner_test_pred.csv $RES_NER_MODEL_DIR/ds_vect.pklz --res_json $PRED_DIR/NER/ner_test_eval.json

#==================RE====================
mkdir -p $PRED_DIR/RE

# Подготовка данных
python re/prepare_ds.py $TRAIN_JSON $PRED_DIR/RE/re_train_data.tsv --mode NER_fixed --save_label_map
python re/prepare_ds.py $TEST_JSON $PRED_DIR/RE/re_test_data_ner_gold.tsv --mode NER_fixed --label_map_path $PRED_DIR/RE/label_map.json

python re/prepare_ds.py $PRED_DIR/NER/ner_test_pred.json $PRED_DIR/RE/re_test_data_ner_predicted.tsv --mode Joint --label_map_path $PRED_DIR/RE/label_map.json


# Обучение RE-модели
python re/fit.py $PRED_DIR/RE/re_train_data.tsv $PRED_DIR/RE/re_train_data.tsv --model_path $RE_MODEL_PATH --model_type $RE_MODEL_TYPE --model_out_path $RES_RE_MODEL_DIR --args_path $RE_MODEL_ARGS

# Предсказания на NER-gold
python re/predict.py $PRED_DIR/RE/re_test_data_ner_gold.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv

# Предсказания на NER-pred
python re/predict.py $PRED_DIR/RE/re_test_data_ner_predicted.tsv --model_path $RES_RE_MODEL_DIR --res_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv

# # Постпроцессинг предсказаний (перевод в .json для оценки)
# Gold
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_gold_ner.tsv --origin_json_file_path $TEST_JSON --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_gold_ner.json

# Pred
python re/save_json_pred.py --data_path $PRED_DIR/RE/re_pred_test_pred_ner.tsv --origin_json_file_path $PRED_DIR/RE/generated_potential_relations.json --label_map_path $PRED_DIR/RE/label_map.json --res_path $PRED_DIR/RE/re_res_pred_pred_ner.json

# Оценка на NER-gold
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_gold_ner.json -o $PRED_DIR/RE/gold_RE_eval_res_
# Оценка на NER-pred
python re/spert_eval.py -t $TEST_JSON -p $PRED_DIR/RE/re_res_pred_pred_ner.json -o $PRED_DIR/RE/pred_RE_eval_res_
'''.strip('\n')

In [74]:
print(sh.replace('python ', '/s/ls4/users/romanrybka/anaconda_relations/envs/relation_extraction_pure/bin/python '))

#! /bin/sh

#SBATCH -D /s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src
#SBATCH -o RDRS_simple_all_folds_%j.out
#SBATCH -e RDRS_simple_all_folds_%j.err
#SBATCH -p hpc5-v100-3d
#SBATCH -n 1
#SBATCH --gres=gpu:v100:1

#TODO:
# Переименовать файлы, чтобы понятные названия были
# Параметры командной строки, а не ворох строк в файле

source /s/ls4/users/romanrybka/anaconda_relations/bin/activate
conda activate /s/ls4/users/romanrybka/anaconda_relations/envs/relation_extraction_pure
which python

#путь до корневой папки проекта
echo "1"
export PYTHONPATH="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/src/"

#datapath
export TRAIN_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/1/train.json"
export TEST_JSON="/s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext_spert_valid_reviews/1/test.json"

# путь к модели изначальной, которую надо fine-tune'ить
export NER_MODEL_PATH="/s/ls4/users/ivan_rus/transformers_models/sagteam/xlm-

In [ ]:
for fold in range(0, 10):
    print(fold)
    for mode in ['train', 'test']:
        with open('./%s/%s.json'%(fold,mode), 'r') as f:
            data = json.load(f)
        for rev in data:
            new_ents = {}
            for i, ent_i in rev['entities'].items():
                if ent_i not in new_ents.values():
                    new_ents[len(new_ents)] = ent_i
            rev['entities'] = new_ents
            for rel in rev['relations']:
                for ent_id, ent in rev['entities'].items():
                    if ent==rel['second_entity']:
                        rel['second_entity']['entity_id'] = ent_id
                        break
                else:
                    raise ValueError
                for ent_id, ent in rev['entities'].items(): 
                    if ent==rel['first_entity']:
                        rel['first_entity']['entity_id'] = ent_id
                        break
                else:
                    raise ValueError